In [1]:
import os
import chardet
from langchain_text_splitters import CharacterTextSplitter, MarkdownHeaderTextSplitter


def load_markdown(data_path):
    with open(data_path, 'rb') as f:
        result = chardet.detect(f.read())
        encoding = result['encoding']

    headers_to_split_on = [
        ("#", "Header 1"),
        ("##", "Header 2"),
        ("###", "Header 3"),
        ("####", "Header 4"),
    ]
    markdown_splitter = MarkdownHeaderTextSplitter(
        headers_to_split_on=headers_to_split_on
    )

    with open(data_path, 'rt', encoding=encoding) as file:
        data_string = file.read()
        documents = markdown_splitter.split_text(data_string)

        # 파일명을 metadata에 추가
        domain = data_path  # os.path.basename(data_path)
        for doc in documents:
            if not doc.metadata:
                doc.metadata = {}
            doc.metadata["domain"] = domain  # Document 객체의 metadata 속성에 파일명 추가

        return documents


def load_txt(data_path):
    with open(data_path, 'rb') as f:
        result = chardet.detect(f.read())
        encoding = result['encoding']

    text_splitter = CharacterTextSplitter(
        separator="\n",
        length_function=len,
        is_separator_regex=False,
    )

    with open(data_path, 'r', encoding=encoding) as file:
        data_string = file.read().split("\n")
        domain = data_path  # os.path.basename(data_path)
        documents = text_splitter.create_documents(data_string)

        for doc in documents:
            if not doc.metadata:
                doc.metadata = {}
            doc.metadata["domain"] = domain  # Document 객체의 metadata 속성에 파일명 추가

        return documents


def load_general(base_dir):
    data = []
    cnt = 0
    for root, dirs, files in os.walk(base_dir):
        for file in files:
            if ".txt" in file:
                cnt += 1
                data += load_txt(os.path.join(root, file))

    print(f"the number of txt files is : {cnt}")
    return data


def load_document(base_dir):
    data = []
    cnt = 0
    for root, _, files in os.walk(base_dir):
        for file in files:
            if ".md" in file:
                cnt += 1
                data += load_markdown(os.path.join(root, file))

    print(f"the number of md files is : {cnt}")
    return data



In [2]:
def get_markdown_files(source_dir):
    md_data = load_document(base_dir=source_dir)
    text_data = load_general(base_dir=source_dir)

    return md_data + text_data



In [3]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from dotenv import load_dotenv
from ragas.testset import TestsetGenerator
from ragas.testset.synthesizers.abstract_query import ComparativeAbstractQuerySynthesizer
# from ragas.testset.synthesizers.base import BaseSynthesizer
# from ragas.testset.synthesizers.base_query import QuerySynthesizer
from ragas.testset.synthesizers.specific_query import SpecificQuerySynthesizer
from ragas.llms import LangchainLLMWrapper

load_dotenv()

generator_llm = ChatOpenAI(model="gpt-4o-mini")
ragas_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o-mini"))
# critic_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o"))
# embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())
generator = TestsetGenerator.from_langchain(generator_llm)
md_files = get_markdown_files(source_dir="/Users/yechanyun/YYC/work/project/AI_Application/rag/data/exynos-ai-studio-docs-main")
test_set = generator.generate_with_langchain_docs(md_files,
                                                  testset_size=1,
                                                  query_distribution=[
                                                    (ComparativeAbstractQuerySynthesizer(llm=ragas_llm), 0.5),
                                                    (SpecificQuerySynthesizer(llm=ragas_llm), 0.5),],
                                                  with_debugging_logs=True)

/Users/yechanyun/YYC/work/project/AI_Application/rag/source/ragtools-all-in-one/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/yechanyun/YYC/work/project/AI_Application/rag/source/ragtools-all-in-one/lib/python3.11/site-packages/ragas/prompt/base.py:9: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from r

the number of md files is : 17
the number of txt files is : 0


Generating Samples: 100%|██████████| 10/10 [00:05<00:00,  1.72it/s]                                                  


In [13]:
test_set.dict()

{'samples': [{'eval_sample': {'user_input': 'comparison of optimization techniques performance evaluations mixed precision quantization static uniform quantization quantizers optimizers',
    'retrieved_contexts': None,
    'reference_contexts': ['The document contains a structured list of topics related to model optimization and analysis, including sections on EHT, converters, optimizers, quantizers, simulators, and performance estimators. It also includes links to quick start guides, system requirements, and usage instructions.',
     'The process involves several steps: converting an ONNX model to CNNX format, optimizing the CNNX model through simplification and dimensional conversion, and evaluating performance before and after optimization. It also includes quantization using methods like Smooth Quantization and Mixed Precision Quantization, followed by performance evaluation of the quantized model. Finally, the CNNX model is converted to SNC format.',
     'The Quantizer module i

In [14]:
evaluation_dataset = test_set.to_evaluation_dataset()
evaluation_dataset
print(evaluation_dataset.dict())

{'samples': [{'user_input': 'comparison of optimization techniques performance evaluations mixed precision quantization static uniform quantization quantizers optimizers', 'retrieved_contexts': None, 'reference_contexts': ['The document contains a structured list of topics related to model optimization and analysis, including sections on EHT, converters, optimizers, quantizers, simulators, and performance estimators. It also includes links to quick start guides, system requirements, and usage instructions.', 'The process involves several steps: converting an ONNX model to CNNX format, optimizing the CNNX model through simplification and dimensional conversion, and evaluating performance before and after optimization. It also includes quantization using methods like Smooth Quantization and Mixed Precision Quantization, followed by performance evaluation of the quantized model. Finally, the CNNX model is converted to SNC format.', 'The Quantizer module in EHT applies both basic and advan

In [25]:
evaluation_dataset.to_jsonl("./evaluation_dataset.jsonl")

In [26]:
test_set.to_jsonl("./testset.jsonl")

In [1]:
from tsk_ragtools import text_gen as tg
from ragas import EvaluationDataset

evaluation_dataset = EvaluationDataset.from_jsonl('./evaluation_dataset.jsonl')
evaluation_dataset_dict = evaluation_dataset.dict()

for i, obj in enumerate(evaluation_dataset_dict["samples"]):
  print(
    f"\rGenerating RAGAS json...({i+1}/{len(evaluation_dataset_dict['samples'])})",
    end="",
    flush=True,
  )
  response = tg.answer_question(obj["user_input"])
  obj.update({"retrieved_contexts": response["list"], "response": response["response"]})

print(evaluation_dataset_dict)

/Users/yechanyun/YYC/work/project/AI_Application/rag/source/ragtools-all-in-one/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/yechanyun/YYC/work/project/AI_Application/rag/source/ragtools-all-in-one/lib/python3.11/site-packages/ragas/prompt/base.py:9: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from r

Generating RAGAS json...(10/10){'samples': [{'user_input': 'comparison of optimization techniques performance evaluations mixed precision quantization static uniform quantization quantizers optimizers', 'retrieved_contexts': ['Section: Quantizer\n# Quantizer\n## Basic Quantization Methods\nContent:\n### Fixed Precision Quantization\nApplies static uniform quantization to both weights and activations. Users can specify the precision (bit-width) for weights and activations. The entire model is quantized to the specified precision\n### Mixed Precision Quantization\nAllows different parts of the model to use different precisions<br>\nTwo approaches are supported:\n- Mixed precision by name: Users specify precisions for specific activation or weight names\n- Mixed precision by operator: Users define precisions for different types of operators. For example, if set the Add operator to INT4 then all outputs of the Add operators are quantized to INT4', 'Section: # Quantizer\n## Basic Quantizati

In [4]:
eval_dataset = EvaluationDataset.from_dict(mapping=evaluation_dataset_dict["samples"])

In [5]:
from ragas.metrics import LLMContextRecall, Faithfulness, FactualCorrectness, SemanticSimilarity
from ragas import evaluate

from ragas.llms import LangchainLLMWrapper
from langchain_openai import ChatOpenAI
evaluator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o-mini"))

metrics = [LLMContextRecall(), FactualCorrectness(), Faithfulness()]
results = evaluate(dataset=eval_dataset, metrics=metrics, llm=evaluator_llm,)

Evaluating: 100%|██████████| 30/30 [00:50<00:00,  1.70s/it]


In [8]:
results.to_pandas().to_dict()

{'user_input': {0: 'comparison of optimization techniques performance evaluations mixed precision quantization static uniform quantization quantizers optimizers',
  1: 'How do quantization methods compare on performance and efficiency?',
  2: 'How do the various model conversion and optimization techniques, including ONNX-to-CNNX conversion, quantization methods, and performance evaluation strategies, compare in their effectiveness and efficiency as outlined in the provided summaries?',
  3: 'How do the various performance evaluation techniques and optimization methods compare across different reports for model optimization, including aspects such as quantization, mixed precision, and the use of specific tools like the Quantizer, Optimizer, and performance estimators?',
  4: 'How do the various optimization techniques and quantization methods for neural networks compare in terms of performance evaluation and model efficiency across different reports?',
  5: 'What parameters need to be 